In [1]:
import pandas as pd
import numpy as np
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data.factset import Fundamentals as FF
from quantopian.pipeline.factors import Returns, Latest, AnnualizedVolatility, MarketCap
from quantopian.pipeline.filters import Q1500US, Q500US, QTradableStocksUS
from quantopian.research import run_pipeline
from statsmodels import regression
import statsmodels.api as sm

**MAIN**

In [2]:
class get_data:
    """Query data"""
    global funda
    
    def __init__(self, start_date, end_date, window):
        self.s = start_date
        self.e = end_date
        self.w = window
        
    def risk_free(self,rf):
        return get_pricing(rf, fields='price', start_date=self.s, end_date=self.e).pct_change(periods=self.w)[1:].rename('r_f')
    
    def market_return(self,rm):
        return get_pricing(rm, fields='price', start_date=self.s, end_date=self.e).pct_change(periods=self.w)[1:].rename('r_m')
    
    def fundamentals(self, funda):

        pipe = Pipeline()

        # get fundamentals and add to pipeline
        for i in funda:
            pipe.add(i[2], i[0])

        # get returns of stocks to pipeline
        pipe.add(Returns(window_length=self.w+1), "Returns")

        #universe
        pipe.set_screen(QTradableStocksUS() & MarketCap().top(1000))

        return run_pipeline(pipe, self.s, self.e)

class mod_data:
    """Modify data"""
    
    def __init__(self, df):
        self.df = df
        
    def add_rank(self, split):
        rd = self.df.copy()
        for i in funda:
            rd['rank_'+ i[0]] = rd[i[0]].groupby(level=0).rank()
            upper = rd['rank_'+ i[0]].max()
            rd['most_'+ i[0]] = (rd['rank_'+ i[0]]>=upper*(1-split))  & (rd['rank_'+ i[0]]<=upper)
            rd['least_'+ i[0]]= (rd['rank_'+ i[0]]< upper*split ) & (rd['rank_'+ i[0]]>=0)
        return rd
    
    def factor_return(self, split):
        rd = self.add_rank(split)
        return pd.concat(
                         [
                          (rd[rd['most_'+ i[0]]]['Returns'].groupby(level=0).mean() -\
                           rd[rd['least_'+ i[0]]]['Returns'].groupby(level=0).mean() \
                           )\
                           .rename('r_'+i[0])\
                            *(-1 if i[1]=='L-H' else 1)\
                            for i in funda\
                         ],\
                          axis=1\
                         )
    def transpose(self):
        x_r= self.df.transpose()
        x_r.index = [i.sid for i in x_r.index]
        return x_r

In [3]:
# Contain the regression methods
class regressor:
    def __init__(self, s_r, f_r, x_r, r_f):
        self.s_r = s_r
        self.f_r = f_r
        self.x_r = x_r
        self.r_f = r_f
        
    # times series regression for a single stock    
    def time_series_linreg(self, X, Y, r_f):
        
        # define na: which will be returned if regression not run
        na = pd.Series([np.nan for i in range(len(X.columns))], index=X.columns)
        
        # if both input are not empty, and inputs contain 50% valid data, run regression
        if not(X.empty or Y.empty) and ( Y.count()/len(Y)>=0.5 ):
            
            try:
                X_1 = sm.add_constant(X)
                model = regression.linear_model.OLS(Y-r_f, X_1, missing='drop').fit()

            except (ValueError,TypeError):
                return na.rename(Y.name.sid)

            return model.params[1:].rename(Y.name.sid)
        else:
            return na.rename(Y.name.sid)
    
    # times series regression for all stock
    def grp_time_series_linreg(self):
        beta = [ self.time_series_linreg(self.f_r, self.s_r.iloc[:,i], self.r_f) for i in range(len(self.s_r.columns))]
        beta = pd.DataFrame(beta)
        beta.columns = ['b_'+ i.split("_")[1] for i in beta.columns]
        return beta
    
    # cross sectional regression for a particular time t
    def x_sec_linreg(self, X, Y, r_f):
        
        # define na: which will be returned if regression not run
        na = pd.Series([np.nan for i in range(len(X.columns))], index=X.columns)
        
        # if both input are not empty, and inputs contain 50% valid data, run regression
        if (not(X.empty or Y.empty)) and (Y.name in r_f.index) and ( Y.count()/len(Y) >= 0.5 ):

            try:
                X_1 = sm.add_constant(X)
                model = regression.linear_model.OLS(Y - r_f[Y.name], X_1, missing='drop').fit()

            except (ValueError,TypeError):
                return na.rename(Y.name)

            return model.params[1:].rename(Y.name)
        else:
            return na.rename(Y.name)
        
    # cross sectional regression for all time t    
    def grp_x_sec_linreg(self):
        beta = self.grp_time_series_linreg()
        r_prem = [self.x_sec_linreg(beta, self.x_r.iloc[:,i], self.r_f) for i in range(len(self.x_r.columns))]
        r_prem = pd.DataFrame(r_prem)
        r_prem.columns = ['rp_'+ i.split("_")[1] for i in r_prem.columns]
        return beta, r_prem.dropna()

In [4]:
class factor_model: 
    def __init__(self, start_date, end_date, factor, rf, rm, window, split):
        self.s   = start_date
        self.e   = end_date
        self.f   = funda
        self.w   = window
        self.sp  = split
        self.rf  = get_data(self.s, self.e, self.w).risk_free(rf)       # riskfree rate: time series
        self.rm  = get_data(self.s, self.e, self.w).risk_free(rm)       # market return: time series
        
    # fundamentals data: cross sectional group by time
        self.df  = get_data(self.s, self.e, self.w).fundamentals(self.f)
        
    # ranked fundamentals
        self.r_d   = mod_data(self.df).add_rank(self.sp)                       
    # Fundamental factor returns: time series 
        self.o_f_r = mod_data(self.df).factor_return(self.sp)
    # All factor returns: time series 
        self.f_r   = pd.concat([(self.rm-self.rf).rename('r_MARKET'),self.o_f_r],axis=1)
    # stock returns:  time series
        self.s_r   = self.df.unstack()['Returns']                       
    # cross sectional stock returns
        self.x_r   = mod_data(self.s_r).transpose()                       
    # Beta: stocks sensitivities to factors
    # risk premia: time series
        self.s_beta, self.r_prem = regressor(self.s_r, self.f_r, self.x_r, self.rf).grp_x_sec_linreg()
    # annualized risk premium
        self.a_r_prem = ((self.r_prem.groupby(self.r_prem.index.year).sum()/252+1)**(252/self.w)-1)

**Test the model for data in 2019**

In [19]:
global funda
funda = [
        #Size(low-high)
        ['SIZE', 'L-H', Latest([Fundamentals.market_cap])**(0.5)],\
        #Min Vol(low-high)
        ['VOL',  'L-H', AnnualizedVolatility(window_length=2)],\
        #value(low-high)
        ['VAL',  'L-H', Latest([Fundamentals.pb_ratio])],\
        #Momentum(high-low)
        ['MMT',  'H-L', (1+Returns(window_length=21*12))/(1+Returns(window_length=12))],\
        #Quality(high-low)
        ['QLT',  'H-L', -0.5*Latest([Fundamentals.total_debt_equity_ratio]).zscore() +\
                         0.5*Latest([Fundamentals.roe]).zscore()]       
        ]

model2019 = factor_model(start_date = '2018-12-02', # Model 1 monthly rolling return during 2019, so we query from 2018Dec
                         end_date   = '2019-12-31', 
                         factor     = funda,        # use the Fundamentals defined above
                         rf         = 'BIL',        # 1-3 month T-Bil ETF as risk free rate
                         rm         = 'SPY',        # S&P500 ETF as market return
                         window     = 21,           # 1 month rolling return
                         split      = 0.3)          # Construct factor returns by top 30% and bottom 30% rank 

In [20]:
# Query fundamentals data
model2019.df.head()

MMT       QLT   Returns  \
2018-12-03 00:00:00+00:00 Equity(2 [HWM])    0.839403  0.019265  0.059172   
                          Equity(24 [AAPL])  1.108862  0.021188 -0.181069   
                          Equity(53 [ABMD])  1.605558  0.020359 -0.028390   
                          Equity(62 [ABT])   1.263445  0.019172  0.073988   
                          Equity(64 [GOLD])  0.910177  0.017978  0.018291   

                                                      SIZE        VAL  \
2018-12-03 00:00:00+00:00 Equity(2 [HWM])    101882.511291   1.956285   
                          Equity(24 [AAPL])  920561.336816   7.909070   
                          Equity(53 [ABMD])  122408.128170  18.896115   
                          Equity(62 [ABT])   358704.555204   4.235677   
                          Equity(64 [GOLD])  122011.533135   1.671361   

                                                  VOL  
2018-12-03 00:00:00+00:00 Equity(2 [HWM])    0.146169  
                          Equity(24 [AAPL])  0.019849  
                          Equity(53 [ABMD])  0.093897  
                          Equity(62 [ABT])   0.037658  
                          Equity(64 [GOLD])  0.209893

In [21]:
# Rank data, Most = Top30%, Least = Bottom30%
model2019.r_d.head()

MMT       QLT   Returns  \
2018-12-03 00:00:00+00:00 Equity(2 [HWM])    0.839403  0.019265  0.059172   
                          Equity(24 [AAPL])  1.108862  0.021188 -0.181069   
                          Equity(53 [ABMD])  1.605558  0.020359 -0.028390   
                          Equity(62 [ABT])   1.263445  0.019172  0.073988   
                          Equity(64 [GOLD])  0.910177  0.017978  0.018291   

                                                      SIZE        VAL  \
2018-12-03 00:00:00+00:00 Equity(2 [HWM])    101882.511291   1.956285   
                          Equity(24 [AAPL])  920561.336816   7.909070   
                          Equity(53 [ABMD])  122408.128170  18.896115   
                          Equity(62 [ABT])   358704.555204   4.235677   
                          Equity(64 [GOLD])  122011.533135   1.671361   

                                                  VOL  rank_SIZE most_SIZE  \
2018-12-03 00:00:00+00:00 Equity(2 [HWM])    0.146169      238.0     False   
                          Equity(24 [AAPL])  0.019849      757.0      True   
                          Equity(53 [ABMD])  0.093897      378.0     False   
                          Equity(62 [ABT])   0.037658      720.0      True   
                          Equity(64 [GOLD])  0.209893      374.0     False   

                                            least_SIZE  rank_VOL    ...     \
2018-12-03 00:00:00+00:00 Equity(2 [HWM])        False     451.0    ...      
                          Equity(24 [AAPL])      False      66.0    ...      
                          Equity(53 [ABMD])      False     322.0    ...      
                          Equity(62 [ABT])       False     130.0    ...      
                          Equity(64 [GOLD])      False     582.0    ...      

                                            least_VOL rank_VAL  most_VAL  \
2018-12-03 00:00:00+00:00 Equity(2 [HWM])       False    200.0     False   
                          Equity(24 [AAPL])      True    601.0      True   
                          Equity(53 [ABMD])     False    699.0      True   
                          Equity(62 [ABT])       True    453.0     False   
                          Equity(64 [GOLD])     False    150.0     False   

                                            least_VAL rank_MMT  most_MMT  \
2018-12-03 00:00:00+00:00 Equity(2 [HWM])        True    121.0     False   
                          Equity(24 [AAPL])     False    495.0     False   
                          Equity(53 [ABMD])     False    732.0      True   
                          Equity(62 [ABT])      False    641.0      True   
                          Equity(64 [GOLD])      True    203.0     False   

                                            least_MMT rank_QLT  most_QLT  \
2018-12-03 00:00:00+00:00 Equity(2 [HWM])        True    219.0     False   
                          Equity(24 [AAPL])     False    668.0      True   
                          Equity(53 [ABMD])     False    591.0      True   
                          Equity(62 [ABT])      False    188.0     False   
                          Equity(64 [GOLD])      True     40.0     False   

                                            least_QLT  
2018-12-03 00:00:00+00:00 Equity(2 [HWM])        True  
                          Equity(24 [AAPL])     False  
                          Equity(53 [ABMD])     False  
                          Equity(62 [ABT])       True  
                          Equity(64 [GOLD])      True  

[5 rows x 21 columns]

In [22]:
# Return difference between the top and bottom groups, with respect to each fundamental factors
model2019.f_r.head()

,r_MARKET,r_SIZE,r_VOL,r_VAL,r_MMT,r_QLT
2018-12-03 00:00:00+00:00,NaN,0.003439,0.009437,-0.034297,0.017168,0.005431
2018-12-04 00:00:00+00:00,NaN,-0.002159,0.023643,-0.034916,0.031939,0.005468
2018-12-06 00:00:00+00:00,NaN,-0.009211,0.067589,-0.029431,0.039216,-0.006677
2018-12-07 00:00:00+00:00,NaN,0.000990,0.001738,-0.044972,0.047262,-0.009038
2018-12-10 00:00:00+00:00,NaN,-0.001488,0.006121,-0.033020,0.046992,-0.017661


In [23]:
# Stock time series return s_r
model2019.s_r.head()

,Equity(2 [HWM]),Equity(24 [AAPL]),Equity(53 [ABMD]),Equity(62 [ABT]),Equity(64 [GOLD]),Equity(67 [ADSK]),Equity(76 [TAP]),Equity(114 [ADBE]),Equity(122 [ADI]),Equity(128 [ADM]),...,Equity(52537 [STNE]),Equity(52553 [SWI]),Equity(52592 [LIN]),Equity(52747 [DELL]),Equity(52751 [MRNA]),Equity(52968 [FOXA]),Equity(52991 [DOW]),Equity(52998 [LEVI]),Equity(53023 [LYFT]),Equity(53046 [TW])
2018-12-03 00:00:00+00:00,0.059172,-0.181069,-0.028390,0.073988,0.018291,0.117911,0.035053,0.020995,0.104135,-0.018203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-04 00:00:00+00:00,0.011278,-0.164761,-0.089841,0.059167,-0.022865,0.095118,0.008392,0.039586,0.068109,-0.031480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-06 00:00:00+00:00,-0.014650,-0.145291,-0.148322,0.019096,-0.001497,0.042652,0.034316,0.026211,0.042862,-0.042347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-07 00:00:00+00:00,-0.011079,-0.130067,-0.152705,0.016049,0.012911,0.049659,0.028268,0.045240,0.042287,-0.046234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-10 00:00:00+00:00,-0.030129,-0.170785,-0.185549,-0.013818,0.045435,-0.008749,0.017824,-0.010600,-0.012101,-0.052493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Regress stock time series return s_r on factor return f_r
# which generates sensitivity towards factors, shown by stock ID on the index column
model2019.s_beta.head()

,b_MARKET,b_SIZE,b_VOL,b_VAL,b_MMT,b_QLT
2,1.115890,-2.382851,-0.275627,0.379287,0.199126,1.319534
24,1.451381,-2.177862,0.159607,-1.304137,-1.110415,-0.778272
53,0.872235,4.219365,-0.406791,0.511601,0.055425,0.062596
62,0.779462,0.556535,-0.011001,-0.354079,0.262172,1.072467
64,1.834765,-3.082367,0.018568,-0.926918,0.421254,-0.066157


In [25]:
# Stock cross sectional return x_r
model2019.x_r.head()

,2018-12-03 00:00:00+00:00,2018-12-04 00:00:00+00:00,2018-12-06 00:00:00+00:00,2018-12-07 00:00:00+00:00,2018-12-10 00:00:00+00:00,2018-12-11 00:00:00+00:00,2018-12-12 00:00:00+00:00,2018-12-13 00:00:00+00:00,2018-12-14 00:00:00+00:00,2018-12-17 00:00:00+00:00,...,2019-12-17 00:00:00+00:00,2019-12-18 00:00:00+00:00,2019-12-19 00:00:00+00:00,2019-12-20 00:00:00+00:00,2019-12-23 00:00:00+00:00,2019-12-24 00:00:00+00:00,2019-12-26 00:00:00+00:00,2019-12-27 00:00:00+00:00,2019-12-30 00:00:00+00:00,2019-12-31 00:00:00+00:00
2,0.059172,0.011278,-0.014650,-0.011079,-0.030129,-0.059130,-0.043230,-0.024320,-0.006972,-0.018399,...,0.052945,0.019760,-0.010296,-0.000645,0.016548,0.029729,0.024223,0.007752,-0.004476,-0.012160
24,-0.181069,-0.164761,-0.145291,-0.130067,-0.170785,-0.189361,-0.191223,-0.173033,-0.120288,-0.139572,...,0.065096,0.055202,0.047132,0.051598,0.062647,0.083814,0.085686,0.088169,0.095434,0.088641
53,-0.028390,-0.089841,-0.148322,-0.152705,-0.185549,-0.214327,-0.207128,-0.184077,-0.053423,-0.043676,...,-0.202571,-0.199266,-0.005870,-0.080883,-0.120513,-0.085540,-0.053242,-0.087603,-0.128225,-0.150262
62,0.073988,0.059167,0.019096,0.016049,-0.013818,-0.021937,-0.015486,-0.003298,0.022524,0.018129,...,0.027220,0.015871,0.031933,0.036189,0.033441,0.043733,0.042772,0.033022,0.023063,0.016155
64,0.018291,-0.022865,-0.001497,0.012911,0.045435,0.068765,0.038975,0.087641,0.125899,0.101164,...,0.048728,0.053850,0.064732,0.041799,0.027444,0.069880,0.112998,0.121097,0.092802,0.113704


In [27]:
# Regress stock cross section return x_r on stock beta s_beta
# which generates risk premium for each time t
model2019.r_prem.head()

,rp_MARKET,rp_SIZE,rp_VOL,rp_VAL,rp_MMT,rp_QLT
2019-01-04 00:00:00+00:00,-0.070681,-0.003816,0.038489,-0.007652,0.020957,0.001166
2019-01-07 00:00:00+00:00,-0.068967,-0.002483,0.031115,-0.008528,0.018722,0.001164
2019-01-08 00:00:00+00:00,-0.053147,0.004558,0.015285,-0.009792,0.001971,0.002716
2019-01-09 00:00:00+00:00,-0.047934,0.006060,0.012438,-0.002851,-0.006789,0.004522
2019-01-10 00:00:00+00:00,-0.032591,0.011449,0.006586,-0.004566,-0.020706,0.005989


In [28]:
#taking average of r_prem and annualize it (since we used 1 month rolling return)
model2019.a_r_prem

,rp_MARKET,rp_SIZE,rp_VOL,rp_VAL,rp_MMT,rp_QLT
2019,0.063178,0.026151,0.050749,-0.122126,0.094184,-0.00612
